In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import glob
import os
import sys
import zipfile

import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

from time import time

import itertools
from sklearn.preprocessing import scale
from sklearn import preprocessing

import boto3
from io import BytesIO
import sagemaker.amazon.common as smac
from sagemaker import get_execution_role

In [2]:
'''
-----------------------------------------------------------------------------
--- Definitions
-----------------------------------------------------------------------------
'''
Results = 'Results'
Indicies = 'Indicies'
Response = 'Response'
Figures = 'Figures'

# Configuring S3
s3_bucket_name = 'jasper-ml-sagemaker'
role = get_execution_role()

client = boto3.client('s3')
resource = boto3.resource('s3')
my_bucket = resource.Bucket(s3_bucket_name)

In [16]:
dataSetName = 'L6polarInterpDataScaled.csv'
dataPath = 'Data/'
dataKey = dataSetName
obj = client.get_object(Bucket=s3_bucket_name, Key=dataPath + dataKey)
dataSet2 = pd.read_csv(obj['Body'], header = None, index_col=None)
dataSet2

,0,1,2,3,4,5,6,7,8,9,...,170,171,172,173,174,175,176,177,178,179
0,-0.011960,0.638875,-0.061355,0.550408,-0.203727,0.505055,-0.141431,0.500064,-0.042372,0.444600,...,-1.219591,1.913972,-1.425895,-1.715184,-1.364262,-1.700215,-1.356145,1.953733,-1.470006,-0.058044
1,0.015952,0.710392,-0.067055,0.567972,-0.169936,0.525586,-0.043135,0.504682,0.051563,0.481701,...,-1.448651,1.847654,-1.629509,-1.675439,-1.452101,-1.758565,-1.405323,1.914641,-1.471291,1.177042
2,0.018551,0.729991,0.018270,0.575248,0.022000,0.539532,0.182010,0.515883,0.195008,0.536356,...,-1.456827,-1.591948,-1.465409,-1.335153,-1.483489,-1.406613,-1.401083,-1.742534,-1.410175,1.643874
3,-0.133622,0.697877,0.029040,0.590249,0.120467,0.574542,0.328148,0.568728,0.259529,0.579054,...,-1.244954,-1.577523,-1.352514,-1.340591,-1.494039,-1.288716,-1.382035,-1.624355,-1.449696,-1.701131
4,-0.241121,0.719393,-0.104877,0.609737,0.082381,0.620634,0.264450,0.613951,0.271767,0.594060,...,-1.307360,-1.628004,-1.502632,-1.563865,-1.511125,-1.707087,-1.372577,-1.615711,-1.423352,-0.904055
5,-0.361699,0.858650,-0.274314,0.667196,0.026628,0.655172,0.143000,0.634963,0.205013,0.637843,...,-1.454022,1.895957,-1.558912,1.936367,-1.479014,1.862470,-1.379106,-1.735044,-1.370321,-1.713791
6,-0.487339,1.084745,-0.319317,0.784620,-0.032551,0.698592,0.203078,0.716548,0.153024,0.747166,...,-1.203458,1.817576,-1.468726,-1.682040,-1.380363,-1.731647,-1.363907,1.794744,-1.264999,-1.670868
7,-0.546960,1.211961,-0.301252,0.850194,-0.027536,0.731361,0.391529,0.809144,0.300131,0.856776,...,-0.850476,1.899145,-1.215194,-1.620405,-1.107628,-1.708729,-1.303454,1.727266,-1.149331,-1.456134
8,-0.699257,1.172017,-0.423968,0.855190,-0.034849,0.732849,0.327904,0.843652,0.355196,0.906857,...,-0.946423,-1.755433,-1.013719,-1.630338,-1.024703,-1.696562,-1.209942,1.853545,-1.044276,-1.549683
9,-0.928573,1.117834,-0.582605,0.896647,-0.120287,0.754417,0.140021,0.864578,0.107721,0.916987,...,-1.218027,-1.754402,-1.043420,-1.674195,-1.070184,-1.638979,-1.093110,-1.720584,-1.055304,-1.671793


,0,1,2,3,4,5,6,7,8,9,...,652,653,654,655,656,657,658,659,660,661
0,0.658465,0.659052,0.547199,0.601384,0.689064,0.838963,0.943719,1.037567,0.898718,1.002651,...,-0.610418,-0.600297,-0.590180,-0.580070,-0.569967,-0.559876,-0.527851,-0.446890,-0.386315,-0.084779
1,0.628274,0.648264,0.572063,0.683673,0.768488,0.948437,0.915408,0.988435,0.911624,1.054714,...,-0.454603,-0.443375,-0.432166,-0.420979,-0.409816,-0.398680,-0.371716,-0.349021,-0.342368,-0.094337
2,0.612895,0.714764,0.728990,0.871317,0.879142,0.924604,0.844561,0.905356,0.846433,0.779876,...,-0.403181,-0.398541,-0.393893,-0.389240,-0.384582,-0.379921,-0.367982,-0.330231,-0.327763,-0.139589
3,0.524568,0.716641,0.798019,0.975126,0.915263,0.894957,0.809106,0.913219,0.818244,0.558805,...,-0.498081,-0.501051,-0.503974,-0.506849,-0.509675,-0.512454,-0.487579,-0.340840,-0.324309,-0.117528
4,0.427586,0.599208,0.742696,0.896834,0.917091,0.914394,0.804697,0.860565,0.800644,0.580361,...,-0.477206,-0.478106,-0.478968,-0.479794,-0.480583,-0.481336,-0.458519,-0.361684,-0.345121,-0.100829
5,0.223714,0.433461,0.675337,0.784098,0.833073,0.788642,0.707575,0.692911,0.762922,0.626414,...,-0.411287,-0.407400,-0.403502,-0.399595,-0.395678,-0.391755,-0.382539,-0.362195,-0.351325,-0.180655
6,-0.092432,0.312859,0.595854,0.761003,0.689476,0.551365,0.588553,0.590565,0.716356,0.627138,...,-0.550439,-0.551189,-0.551898,-0.552565,-0.553192,-0.553779,-0.522476,-0.426829,-0.370054,-0.263578
7,-0.266446,0.262075,0.571008,0.780126,0.640936,0.477323,0.546409,0.570692,0.640963,0.614446,...,-0.751001,-0.760935,-0.770773,-0.780512,-0.790152,-0.799693,-0.752946,-0.600820,-0.445310,-0.328601
8,-0.261239,0.178379,0.564144,0.682887,0.587719,0.488373,0.478881,0.503596,0.559365,0.490644,...,-0.676584,-0.688077,-0.699470,-0.710763,-0.721954,-0.733043,-0.734376,-0.695065,-0.523931,-0.423889
9,-0.289943,0.042611,0.480663,0.527734,0.422469,0.342985,0.385410,0.461649,0.524491,0.327330,...,-0.525559,-0.530459,-0.535300,-0.540083,-0.544806,-0.549469,-0.587747,-0.669470,-0.581228,-0.429559
